In [89]:
import pandas as pd
import os

In [90]:
weapons_data = pd.read_csv('eldenringScrap/weapons.csv')

In [91]:
# def table_to_text(df, id_col, drop_cols):
#     df = df.drop(columns=drop_cols)
#     df_melted = df.melt(id_vars=id_col)
#     assert len(df_melted) == len(df) * (len(df.columns) - 1)
#     df_melted['value'] = df_melted['value'].astype(str)
#     text_df = pd.DataFrame(columns=['text'])
#     text_df['text'] = 'The ' + df_melted['variable'] + ' of ' + df_melted['name'] + ' is ' + df_melted['value']
#     return text_df


In [92]:
# weapons.csv
# weapons_txt = table_to_text(weapons_data, 'name', ['id', 'weapon_id', 'image'])
# weapons_txt.to_csv('weapon_txt.csv', index=False)


In [93]:
# print(weapons_txt.head())

In [94]:
def get_dfs(path: str):
    dfs = []
    fnames = []
    for file in os.listdir(path):
        if not file.endswith('.csv'):
            continue
        fnames.append(file)
        df = pd.read_csv(f'{path}/{file}')
        if 'image' in df.columns:
            df = df.drop(columns=['image'])
        if 'dlc' in df.columns:
            df = df.drop(columns=['dlc'])
        if 'id' in df.columns:
            df = df.drop(columns=['id'])
        if 'shield_id' in df.columns:
            df = df.drop(columns=['shield_id'])
        if 'weapon_id' in df.columns:
            df = df.drop(columns=['weapon_id'])
        if 'shield name' in df.columns:
            df = df.rename(columns={'shield name': 'name'})
        if 'weapon name' in df.columns:
            df = df.rename(columns={'weapon name': 'name'})
        if 'value' in df.columns:
            # print(file), only talismans.csv and remembrances.csv have 'value'
            df = df.rename(columns={'value': 'sell_value'})
        dfs.append(df)
    return dfs, fnames


In [95]:
list1, fnames1 = get_dfs('eldenringScrap')
list2, fnames2 = get_dfs('eldenringScrap/items')
all_dfs = list1 + list2
all_fnames = fnames1 + fnames2

In [96]:
# find shields_upgrades and weapons_upgrades
shield = all_fnames.index('shields_upgrades.csv')
weapon = all_fnames.index('weapons_upgrades.csv')
print(shield, weapon)
shield_df, weapon_df = all_dfs[shield], all_dfs[weapon]
shield_df['name'] = shield_df['name'] + ' with upgrade ' + shield_df['upgrade']
weapon_df['name'] = weapon_df['name'] + ' with upgrade ' + weapon_df['upgrade']
shield_df = shield_df.drop(columns=['upgrade'])
weapon_df = weapon_df.drop(columns=['upgrade'])
all_dfs[shield], all_dfs[weapon] = shield_df, weapon_df

3 8


In [97]:
for df, fname in zip(all_dfs, all_fnames):
    print(fname)
    print(df.columns)
    print()

npcs.csv
Index(['name', 'location', 'role', 'voiced by', 'description'], dtype='object')

spiritAshes.csv
Index(['name', 'type', 'FP cost', 'HP cost', 'effect', 'description'], dtype='object')

creatures.csv
Index(['name', 'locations', 'drops', 'blockquote'], dtype='object')

shields_upgrades.csv
Index(['name', 'attack power', 'stat scaling', 'passive effects',
       'damage reduction (%)'],
      dtype='object')

skills.csv
Index(['name', 'type', 'equipament', 'charge', 'FP', 'effect', 'locations'], dtype='object')

locations.csv
Index(['name', 'region', 'items', 'npcs', 'creatures', 'bosses',
       'description'],
      dtype='object')

shields.csv
Index(['name', 'weight', 'description', 'requirements', 'damage type',
       'category', 'passive effect', 'skill', 'FP cost'],
      dtype='object')

talismans.csv
Index(['name', 'effect', 'weight', 'sell_value', 'description'], dtype='object')

weapons_upgrades.csv
Index(['name', 'attack power', 'stat scaling', 'passive effects',
    

In [98]:
def table_to_text(df):
    df_melted = df.melt(id_vars='name', value_vars=df.columns[1:])
    assert len(df_melted) == len(df) * (len(df.columns) - 1)
    # drop nan in value
    df_melted = df_melted.dropna(subset=['value'])
    df_melted['value'] = df_melted['value'].astype(str)
    text_df = pd.DataFrame(columns=['text'])
    text_df['text'] = 'The ' + df_melted['variable'] + ' of ' + df_melted['name'] + ' is ' + df_melted['value']
    return text_df


In [99]:
# save in separate files
# os.makedirs('text_data', exist_ok=True)
# save_path = 'text_data'
# for df, fname in zip(all_dfs, all_fnames):
#     text_df = table_to_text(df)
#     text_df.to_csv(f'{save_path}/{fname[:-4]}_txt.csv', index=False)


In [100]:
df_all = pd.concat(all_dfs)
text_df_all = table_to_text(df_all)
text_df_all.to_csv(f'kaggle_data.txt', index=False, header=False)